Hypothetical interaction

In [19]:
import sys
import os

# Add actrme to path
sys.path.append(os.path.abspath(os.path.join(os.path.abspath("."), '..', 'actrme')))
from importlib import reload
import actrme.model
from actrme.modules.declarative import DeclarativeMemory
import pandas as pd


# Examples

Here is a list of examples.

## 1. A Memory Experiment

We will start with a memory experoiment. The experiment is taken from a paper by Xu et al. (2025), and the original data is available on OSF.

The experiment used a retrieval practice procedure: Participants are presented with simple paired associates, e.g., a pair of Swahili-English words, such as "samaki/fish". Each pair is presented during a study trial, and then probed in a series of test trials.

In [39]:
data = pd.read_csv("../data/memory_experiment.csv")
data

,subj,item,resp,RT,type,time,acc
0,1501,3,fish,2.295,study,3.131,1
1,1501,3,fish,0.888,test,5.426,1
2,1501,12,book,3.128,study,10.248,1
3,1501,12,book,0.841,test,12.776,1
4,1501,2,beer,4.203,study,18.824,1
...,...,...,...,...,...,...,...
146,1501,10,cloud,2.137,test,712.918,1
147,1501,5,office,1.025,test,715.292,1
148,1501,12,book,0.975,test,717.400,1
149,1501,16,work,1.042,test,719.503,1


This is an alteration of "study" and "test" trials. 

### Creating a model

To fit the data, we need to create a simple model. Fitting the data means that the model is run trial by trial to maximize the probability of responding exactly like the participant.

To do so, we need to create a `DataModel`, which is a generic name for a model simulates behavior or performance "tracing".

In [42]:
reload(actrme.model)
model = actrme.model.DataModel(data)
model

Now, we have not yet specified how the model works. In this example, we will create a simple model that does only one thing: encodes things on study trials, and attempts to retrieve them on test trials. To do so, we need to add a new cognitive function to the model, Declarative Memory. 

In [43]:
dm = DeclarativeMemory()
print(dm)
model.add_module(dm)

<GenericModule [module]>


In [44]:
model.outputs


In [45]:

# DM has 3 inputs (encode, time, retrieve), so all of these inputs are 
# now part of the model's inputs.
model.inputs()

# This should return
#    > model.inputs()
#    > ["encode", "time", "retrieve"]
# 

# DM has 2 outputs (response, rt), so all of these outputs are 
# now part of the model's outputs.
model.outputs()

# This should return
#    > model.outputs()
#    > ["rt", "response", "response_probability", "rt_probability"]
# 

# Now, connect the model to the file

model.connect_input("encode", data.study)
model.connect_input("time", data.time)
model.connect_input("retrieve", data.cue)

# We want to simply maximize the probability of a response

model.connect_output("probability", data.acc)

# Now fit:
model.fit()


TypeError: 'list' object is not callable